In [124]:
import pandas as pd
import numpy as np

# dataset = "RP-Mod-Crowd"
mod_crowd_df = pd.read_csv("./Datasets/RP-Mod-Crowd.csv")
mod_folds_df = pd.read_csv("./Datasets/RP-Mod-folds.csv")
mod_crowd_df.rename(columns = {"Text":"text", "Reject Newspaper": "label"}, inplace=True)
mod_crowd_df = mod_crowd_df[["text", "label"]]

# df[list(df.text.str.contains("Kurden"))]
def count_occurences(df, word, label_attr="label", text_attr="text"):
    occ = df[np.array(df[text_attr].str.contains(word), dtype=bool)]
    num_pos = len(occ.loc[occ[label_attr] == 1].drop_duplicates())
    num_neg = len(occ.loc[occ[label_attr] == 0].drop_duplicates())
    return num_pos, num_neg
# count_occurences(df, "Kurden", "Reject Newspaper")
word = "Türkei"
print(count_occurences(mod_folds_df, word))
print(count_occurences(mod_crowd_df, word))


(34, 27)
(33, 287)


### Get Important SHAP words and filter

In [114]:
top_200_shap = [' Migranten',
 ' Nazi',
 'arme',
 'Muslim',
 'monster   ',
 'ackt',
 ' Psychiater',
 'sy',
 ' Juden',
 'gewalt',
 'flüchtling',
 ' Männer',
 ' Muslime',
 'M',
 'täter',
 ' Typen',
 ' Mann',
 ' Öko',
 'äh',
 ' Radikal',
 'heit',
 ' Virus',
 'geschützt ',
 ' Kur',
 'Ideologie',
 ' Lebewesen',
 'gewandert',
 ' Terrorismus',
 ' DE',
 ' Mund',
 'ler',
 ' islamische',
 'gesperrt',
 'ten',
 ' Schweine',
 ' In',
 'staaten',
 'brecher',
 'fisch',
 ' Asyl',
 ' Rechts',
 ' Rechten',
 'hirn',
 ' Frau',
 'DW',
 ' Demo',
 ' fremden',
 ' Türkei',
 ' Youtube',
 ' Auge',
 'Hö',
 'ierer',
 ' hilflos',
 'deutsche',
 ' Flüchtlings',
 ' Oma',
 'eten',
 'Krieg',
 'din',
 ' Opfer',
 ' Familie ',
 ' dir ',
 ' intelligente',
 ' Falsch',
 'cke',
 'stern',
 ' Putin',
 'Na',
 'feindlichkeit ',
 ' Islam',
 ' Maul',
 'den',
 'lein',
 ' christlichen',
 ' dieses',
 ' Anti',
 ' Einwanderung',
 'NE',
 'flotte',
 ' deutsche',
 'frei',
 'tuch',
 'EI',
 ' seit',
 ' Täter',
 ' homo',
 ' Vornamen',
 'Veteran',
 ' Sozial',
 ' direkt',
 ' Polizisten',
 ' nehmt',
 'offen',
 ' weiße',
 'komp',
 ' Versa',
 'PE',
 ' ganzen',
 ' ver',
 ' diesem',
 ' Ex',
 'üll',
 'feindlich',
 ' kleines',
 'legal',
 ' Kanzlerin',
 ' Migrations',
 ' eure',
 ' KO',
 ' deutschen',
 ' armen',
 ' links',
 ' sitzt',
 ' Schmier',
 'noch',
 'extrem',
 ' Links',
 'Bruder',
 ' Schä',
 ' Sel',
 'den ',
 ' Namen',
 ' Ihren',
 ' blind',
 ' schwarz',
 'gekippt',
 ' Elite',
 ' Dia',
 ' DANKE',
 ' Braun',
 ' deutschland',
 ' Wo',
 ' sitzen',
 ' Aktivist',
 'ausbeute',
 ' zufällig',
 ' Bon',
 ' Haufen',
 ' Europa',
 'rr',
 'ah',
 'schieben ',
 'denken',
 'ph',
 ' pärchen',
 'Politiker',
 'affe',
 'te',
 ' https',
 ' Person',
 ' bekommt',
 'gehalten',
 'VA',
 ' wer',
 'macher',
 'hintergrund',
 'zel',
 'se',
 ' NRW ',
 ' egal',
 'lk',
 ' L',
 ' Integrations',
 ' meint',
 'arzt',
 'ach',
 ' Ver',
 'wall',
 ' Merkel',
 ' jüdischen',
 ' verrückt',
 ' Hass',
 ' Dame',
 'haftung',
 ' Migration',
 ' Geisel',
 ' Ihnen',
 'ischen',
 ' Demonstranten',
 ' voll',
 'unt',
 ' Mensch',
 ' Jugend',
 'bl',
 ' wohnen',
 'N ',
 ' bitte',
 'journal',
 'r',
 ' Hirn',
 ' schreibt',
 ' jemanden ',
 ' AfD',
 ' tötet',
 ' richtig',
 'Mich',
 ' Anonymität',
 ' deutschland ',
 ' Landes',
 ' könnte']

In [116]:
#TODO: eliminate all words which are not underrepresented
#TODO: eliminate all words which are subsequences of other words
#TODO: further filters or manual elimination??

def underrepresented(word):
    pos, neg = count_occurences(mod_folds_df, word)
    if pos > neg:
        return True
    return False 
shap_underrepresented = list(filter(underrepresented, top_200_shap))
len(shap_underrepresented)

105

In [117]:
shorts = list(filter(lambda word: len(word) < 3, shap_underrepresented))
print([count_occurences(mod_crowd_df, s) for s in shorts])
print([count_occurences(mod_folds_df, s) for s in shorts])

[(204, 1342), (2, 8), (67, 749), (66, 525), (52, 575), (7, 74)]
[(203, 135), (2, 1), (66, 65), (66, 55), (52, 44), (7, 6)]


## Create a dataset which has even distributions of all the words in given list

In [79]:
def find_all_with_word(df, word, text_attr="text"):
    occ = df[np.array(df[text_attr].str.contains(word), dtype=bool)]
    return occ

def get_ex_with_word(df, word, text_attr="text", label_attr="label", sign=0):
    occ = df[np.array(df[text_attr].str.contains(word), dtype=bool)]
    return occ.loc[occ[label_attr] == sign]

def none_of_the_words(df, word_list, text_attr="text"):
    arr = np.array([np.array(df[text_attr].str.contains(word), dtype=bool) for word in word_list])
    bools = np.all(~arr, axis=0)
    assert len(bools) == len(df)
    return df[bools]

def at_least_one_word(df, word_list, text_attr="text"):
    arr = np.array([np.array(df[text_attr].str.contains(word), dtype=bool) for word in word_list])
    bools = np.any(arr, axis=0)
    assert len(bools) == len(df)
    return df[bools]

def undersample_evenly(pos_df, neg_df, word_list):
    #TODO: the point was to add more from the other df, this will just keep the dataframe the same
    # for word in word_list 
    # indices = neg_df.index
    chosen = []
    for word in word_list:
        pos_with_word_df = find_all_with_word(pos_df, word)
        num_samples = len(pos_with_word_df)

        neg_with_word_indices = find_all_with_word(neg_df, word).index
        # print(neg_with_word_indices)
        if num_samples >= len(neg_with_word_indices):
            chosen.extend(neg_with_word_indices)
        else:
            ind = np.random.choice(neg_with_word_indices, num_samples)
            chosen.extend(ind)

    chosen_df = neg_df.loc[chosen]

    pos_wo_words = none_of_the_words(pos_df, word_list)
    neg_wo_words = none_of_the_words(neg_df, word_list)

    ind = np.random.choice(len(neg_wo_words), len(pos_wo_words))

    wo_words_undersampled_df = neg_wo_words.iloc[ind]

    combined_df = pd.concat([pos_df, chosen_df, wo_words_undersampled_df])

    return combined_df

### Remove Data Leak from Validation and Train Datasets

In [126]:
mod_folds_df = pd.read_csv("./Datasets/RP-Mod-folds.csv")
train_folds_df = mod_folds_df.loc[mod_folds_df["ten_folds"] < 8]
valid_folds_df = mod_folds_df.loc[mod_folds_df["ten_folds"] >= 8]
merged = pd.merge(train_folds_df, valid_folds_df, how="inner", on=["text"])[["text", "train_test_split_x", "train_test_split_y", "label_x", "label_y"]]
duplicate_text_lst = list(merged["text"])

In [127]:
train_folds_df = mod_folds_df.loc[mod_folds_df["ten_folds"] < 8]
valid_folds_df = mod_folds_df.loc[mod_folds_df["ten_folds"] >= 8]

leaks_removed_df = valid_folds_df.copy(deep=True)
for ind in valid_folds_df.index:
    if leaks_removed_df.loc[ind, "text"] in duplicate_text_lst:
        leaks_removed_df = leaks_removed_df.drop(ind)

### Undersample the datasets
1. validation dataset with added words
2. small dataset with just the false positives words, for easier assessment of the techniques
3. new training dataset
4. total dataset with training and validation dataset concatenated together :) 

In [128]:
#TODO: make columns line up better :) 
neg_total_df = mod_crowd_df.loc[mod_crowd_df["label"] == 0]
pos_valid_df = leaks_removed_df.loc[leaks_removed_df["label"] == 1]
pos_train_df = train_folds_df.loc[train_folds_df["label"] == 1]
# undersample_evenly(train_folds_df, small_word_list)

In [136]:
#TODO: create three datasets: 
# 1. a small dataset with just the false positive words in the validation dataset 
# 2. validation dataset with added words (a superset of the dataset)
# 3. a training dataset without the examples which were added to the validation dataset in steps 1 & 2

new_valid_df = undersample_evenly(pos_valid_df, neg_total_df, shap_underrepresented)
new_valid_df["train_test_split"] = np.ones(len(new_valid_df))
small_words_df = at_least_one_word(new_valid_df, shap_underrepresented)

indices = neg_total_df.index
neg_in_valid = new_valid_df.loc[new_valid_df["label"] == 0]
neg_not_in_valid_indices = list(set(indices) - set(neg_in_valid.index))

neg_not_in_valid_df = neg_total_df.loc[neg_not_in_valid_indices]

new_train_df = undersample_evenly(pos_train_df, neg_not_in_valid_df, shap_underrepresented)
new_train_df["train_test_split"] = np.zeros(len(new_train_df))

In [120]:
print(len(train_folds_df), len(new_train_df), len(leaks_removed_df), len(new_valid_df))

11426 13001 2830 3185


In [141]:
combined_df = pd.concat([new_train_df, new_valid_df])
combined_df = combined_df[["text", "label", "train_test_split"]]
combined_df

,text,label,train_test_split
0,Das hellste Licht ist der nun wahrlich nicht.,1.0,0.0
2,In wievielen türkischen oder arabischen Firmen...,1.0,0.0
4,AKK und KK. Alles nur noch Kakka.,1.0,0.0
6,Gladbacher - 100 % Zustimmung. Ich finde diese...,1.0,0.0
8,gähn,1.0,0.0
...,...,...,...
49649,Haben wir keine wichtigen Probleme und Themen ...,0.0,1.0
78503,Yeah! Macht die Straße endlich wieder auf! Wei...,0.0,1.0
17886,""""" Bald könnte es den Big Mac nicht nur bei Mc...",0.0,1.0
23153,"Bei diesen ""Antidiskriminierungsbeauftragten"" ...",0.0,1.0


### Save the datasets to the appropriate folders

In [88]:
print(len(train_folds_df))
no_duplicates = train_folds_df.drop_duplicates()
print(len(no_duplicates))

11426
11415


In [90]:
def compare_with_no_duplicates(df):
    no_duplicates = df.drop_duplicates()
    return len(df) - len(no_duplicates)
print(compare_with_no_duplicates(mod_crowd_df))
print(compare_with_no_duplicates(mod_folds_df))

0
11


In [108]:
# checking if overlap between original valid and train dataset
#TODO: how did the duplicates get there??
mod_folds_df = pd.read_csv("./Datasets/RP-Mod-folds.csv")
train_folds_df = mod_folds_df.loc[mod_folds_df["ten_folds"] < 8]
valid_folds_df = mod_folds_df.loc[mod_folds_df["ten_folds"] >= 8]
print(len(valid_folds_df))
merged = pd.merge(train_folds_df, valid_folds_df, how="inner", on=["text"])[["text", "train_test_split_x", "train_test_split_y", "label_x", "label_y"]]
duplicate_text_lst = list(merged["text"])

2856


In [109]:
leaks_removed_df = valid_folds_df.copy(deep=True)
for ind in valid_folds_df.index:
    if leaks_removed_df.loc[ind, "text"] in duplicate_text_lst:
        leaks_removed_df = leaks_removed_df.drop(ind)

In [110]:
len(valid_folds_df), len(leaks_removed_df)

(2856, 2830)

In [81]:
#TODO: check that there is no overlap between the validation and train datasets :) 
merged = pd.merge(new_validation_df, new_train_df, how="inner", on=["text"])
merged

,text,label_x,five_folds_x,ten_folds_x,train_test_split_x,Unnamed: 0.1_x,Unnamed: 0_x,id_x,Reject Crowd_x,Rejection Count Crowd_x,...,id_y,Reject Crowd_y,Rejection Count Crowd_y,Sexism Count Crowd_y,Racism Count Crowd_y,Threat Count Crowd_y,Insult Count Crowd_y,Profanity Count Crowd_y,Meta Count Crowd_y,Advertisement Count Crowd_y
0,Eine höchst parteiliche Figur als Bundespräsid...,1.0,4.0,8.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Vielleicht sollte Carola mal bei ihrem Waffens...,1.0,4.0,8.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Egal. was Merkels letzter Fan wichtiges von si...,1.0,4.0,8.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"""Dabei ließ sich Borussia auch nach einer Viel...",1.0,4.0,8.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Verlogen wie die alle sind kann man nur Kotzen,1.0,4.0,8.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Lagerfeuer in´der Badewane. Ja, nee ist klar. ...",1.0,4.0,8.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,123,1.0,4.0,8.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Die IG Kö hat offensichtlich die Zeichen der Z...,1.0,4.0,8.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Die IG Kö hat offensichtlich die Zeichen der Z...,1.0,4.0,9.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Evelyn wer?,1.0,4.0,8.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
len(train_folds_df), len(valid_folds_df)

(11426, 2856)

In [40]:
arr = np.array([True, True])
~arr

array([False, False])

In [ ]:
# get list of group identifiers in German which have strong influence on predictions
lst = ["Schwarzer", "Kurden", "Polen", "Migration", "Frau", "Ausland", "Kopftuch", "Juden", "Merkel", "Flüchtling"]
# make sure that there is an equal number of accepted and rejected examples for each

In [68]:
def find_all_with_word(df, word, text_attr="text"):
    occ = df[np.array(df[text_attr].str.contains(word), dtype=bool)]
    return occ

def get_label_group(input_df, label_attr="label"):
    return input_df.loc[input_df[label_attr] > 0]

def compare_positives(pos_arr1, pos_arr2):
    in1not2 = [el for el in pos_arr1 if not el in pos_arr2]
    in2not1 = [el for el in pos_arr2 if not el in pos_arr1]
    return in1not2, in2not1

pos_word_df1 = get_label_group(find_all_with_word(mod_crowd_df, "Kurden", "Text"), "Reject Newspaper")
pos_word_df2 = get_label_group(find_all_with_word(mod_folds_df, "Kurden"))
compare_positives(pos_word_df1.Text.to_numpy(), pos_word_df2.text.to_numpy())

([nan, nan], [nan, nan])

In [42]:
mod_rejected = np.sum(mod_crowd_df["Reject Newspaper"])
mod_rejected

7141

In [63]:
mod_crowd_df.loc[mod_crowd_df["Reject Newspaper"] > 0]

,Unnamed: 0.1,Unnamed: 0,id,Text,Reject Newspaper,Reject Crowd,Rejection Count Crowd,Sexism Count Crowd,Racism Count Crowd,Threat Count Crowd,Insult Count Crowd,Profanity Count Crowd,Meta Count Crowd,Advertisement Count Crowd
49,49,49,1911443,das alles ist wie Selbstbefriedigung...denn sc...,1,1,4,1.0,0.0,0.0,2.0,4.0,0.0,0.0
111,111,111,1911699,Na Gott sei Dank!!! Wen soll er denn auch besc...,1,0,2,0.0,0.0,0.0,1.0,1.0,1.0,0.0
172,172,172,1911995,"Naja, er kann sich ja schlecht daneben stellen...",1,0,2,0.0,0.0,0.0,1.0,0.0,1.0,0.0
193,193,193,1912109,Oh Gott - jetzt lässt Kellermann auch noch den...,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
289,289,289,1912503,Mal gespannt. als Ausrede warum er niemals be...,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84995,84995,84995,2522729,Mein Gott!! Was für ein entsetzliches Gequatsc...,1,0,2,2.0,0.0,0.0,1.0,0.0,0.0,0.0
84996,84996,84996,2522757,Der hat halt auch nichts in der hohlen Birne!!!!!,1,1,3,0.0,0.0,0.0,3.0,0.0,0.0,0.0
84997,84997,84997,2522763,"Blabla, nehme das Auto...",1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84998,84998,84998,2522959,sack reis in china umgefallen...,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
rejected_original = get_label_group(mod_crowd_df, "Reject Newspaper")
rejected_new = get_label_group(mod_folds_df)
compare_positives(rejected_original.Text.to_numpy(), rejected_new.text.to_numpy())

([],
 ['In wievielen türkischen oder arabischen Firmen arbeiten Deutsche?  In Düsseldorf gibt es ein  russisches Geschäft,  in den nur Russen und Russinnen beschäftigt sind. Der "Muslim-Markt" im Internet hat strikte Vorstellungen von MitarbeiterInnen.  Dort sollte an sich mal die Stellenangebote ansehen......Inhaber sind übriges 2 Brüder  der ehemaligen Integrations - Beauftragten Özogüz... ',
  '... und gleichzeitig werden mit Genehmigung der friedfertigen und menschenrechtspropagierenden Bundesregierung welche (Kriegs-)Waffen in welchem Wert in welche Länder exportiert? Und was ist mit unseren Soldaten, die unser Parlament, unsere MdBs ganz legal in außerhalb des NATO-Territoriums gelegene Kriegsgebiete entsandt hat - gehören die nicht allesamt unverzüglich zurück in heimische Gefilde geholt?',
  'Das ist ungefähr so, als ob Goebbels die Nürnberger Reichsparteitagsteilnehmer mit der "Internationale" aufgeputscht hätte, weil das "Horst-Wessel-Lied" keine emotionale Wirkung gezeigt ha

In [ ]:
np.sum()

In [49]:
len(rejected_original), len(rejected_new)

(4632, 7128)

In [44]:
mod_rejected = np.sum(mod_folds_df["label"])
mod_rejected

7141.0

In [39]:
pos_word_df1["Text"].iloc[0]

' Kurden und Linke stürmen WDR-Studio https://www.nw.de/lokal/bielefeld/mitte/22352586_Polizeieinsatz-Demonstranten-stuermen-Bielefelder-WDR-Studio.html'

In [33]:
# investigate the heldout dataset
# this is a dataset of accepted comments from the original dataset for later testing??
# or just the ones that didn't make it in...
arr = np.load("./Datasets/abusive_moderators_holdout.npy", allow_pickle=True)
arr.shape

(70718,)